# Regra do Trapézio

In [1]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

# Regra do Trapézio

In [2]:
def trapezio(start: float, end: float, func: Callable, N: int, final: bool = False) -> (float | tuple[np.ndarray, np.ndarray]):
    """
        Calcula o integral de `func` de `start` a `end` com N passos usando a regra do trapézio. Devolve apenas o valor final se `final == True`.
    """

    # Abcissas
    h = (end - start) / N
    tt = np.linspace(start, end, N + 1)
    vv = func(tt)

    # Área dos trapézios
    aa = (vv[:-1] + vv[1:]) / 2
    integral = h * np.concatenate((np.array([0]), aa.cumsum()))

    if final:
        return integral[-1]

    else:
        return tt, integral

# Regra do Trapézio Adaptativa

In [3]:
def trapezioA(start: float, end: float, func: Callable, eps: float = 1e-6, maxN: int = 10_000) -> float:
    """
        Calcula o integral de `func` de `start` a `end` usando a regra do trapézio adaptativa com no máximo `maxN` fatias.
    """

    # Integral para N = 1
    N1 = 1
    h1 = (end - start) / N1
    I1 = h1 * (func(start) + func(end)) / 2

    # Usar o método adaptativo
    erro = eps + 1
    while abs(erro) > eps:
        # Calcular o próximo integral
        N2 = N1 * 2
        h2 = h1 / 2
        xx = start + np.arange(1, N2, 2) * h2
        ff = func(xx)

        I2 = I1 / 2 + h2 * ff.sum()

        # Estimar o erro
        erro = (I2 - I1) / 3

        # print(f"Para N = {N2} o valor do integral é {I2} e o erro é cerca de {erro}")

        # O I2 atual é o próximo I1
        N1, h1, I1 = N2, h2, I2

        if N1 > maxN:
            raise ValueError(f"Não foi possível calcular o integral com a precisão desejada com menos de maxN = {maxN} fatias!")
    
    return I1

# Método de Romberg com Regra do Trapézio

Neste programa estamos a usar o truque que desenvolvemos no método adaptativo para calcular integrais com o dobro dos passos para otimizar o cálculo de integrais!

In [4]:
def rombergT(start: float, end: float, func: Callable, eps: float = 1e-6, N1: int = 1, maxN: int = 10_000) -> float:
    """
        Calcula o integral de `func` de `start` a `end` usando integração de Romberg com base na regra do trapézio começando com `N1` fatias e usando no máximo `maxN` fatias.
    """

    # Matriz com as últimas duas linhas da matriz R_{i, j}
    RRnow = []
    RRprev = []

    # Calcular R_{0, 0}
    h1 = (end - start) / N1
    RRprev += [h1 * (func(start) + func(end)) / 2]

    
    # Queremos calcular R_{i, i}
    i = 1

    # Usar o método de Romberg
    erro = eps + 1
    while abs(erro) > abs(eps):
        # O número de passos que damos duplica
        N2 = N1 * 2
        h2 = h1 / 2

        # Os novos passos estão a meio dos passos anteriores
        xx = start + np.arange(1, N2, 2) * h2
        ff = func(xx)

        # Calcular R_{i, 1} usando o resultado guardado em R_{i-1, 0}
        RRnow += [RRprev[0] / 2 + h2 * ff.sum()]


        # Calcular R_{i, j+1}, pelo que j+1 vai de 1 até i 
        for j in range(0, i):
            RRnow += [RRnow[j] + (RRnow[j] - RRprev[j]) / (4**(j+1) - 1)]
        
        # Estimar o erro de R_{i, i}
        erro = (RRnow[i-1] - RRprev[i-1]) / (4**i-1)

        # print(f"Para N = {N2} o valor do integral é {RR[i][i]} e o erro é cerca de {erro}")

        # O I2 atual é o próximo I1
        N1, h1 = N2, h2
        RRprev, RRnow = RRnow.copy(), []

        # Aumentar i
        i += 1

        if N1 > maxN:
            raise ValueError(f"Não foi possível calcular o integral com a precisão desejada com menos de maxN = {maxN} fatias!")
    
    return RRprev[i-1]

# Regra do Trapézio 2D

In [5]:
def trapezio2d(xlim: tuple[float, float], ylim: tuple[float, float], func: Callable, N: tuple[int, int]) -> (float | tuple[np.ndarray, np.ndarray]):
    """
        Calcula o integral de `func(x, y)` no bloco onde x está no intervalo `xlim` e y no intervalo `ylim` com `N[0]` passos segundo x e `N[1]` passos segundo y usando a regra do trapézio.
    """
    
    # Verificar que N é par
    Nx, Ny = N

    # Abcissas
    hx = (xlim[1] - xlim[0]) / Nx
    hy = (ylim[1] - ylim[0]) / Ny

    # Grelha de pontos
    xx = np.linspace(xlim[0], xlim[1], Nx + 1)
    yy = np.linspace(ylim[0], ylim[1], Ny + 1)

    # Pesos do Trapézio
    wx = np.ones_like(xx)
    wx[1:Nx] = 2
    wx *= hx / 2

    wy = np.ones_like(yy)
    wy[1:Ny] = 2
    wy *= hy / 2

    # Meshgrid da grelha e dos pesos
    xm, ym = np.meshgrid(xx, yy, sparse=True)
    wxm, wym = np.meshgrid(wx, wy, sparse=True)

    # Calcular o integral
    return np.sum(func(xm, ym) * wxm * wym)

# Testar

Vamos fazer uma bateria de testes a este métodos!

In [6]:
from testes import testeintegral

In [7]:
print("Regra do Trapézio\n")
testeintegral(lambda start, end, func, N: trapezio(start, end, func, N, True), 10)

Regra do Trapézio

Integral de x de 0 a 1
Valor obtido: 0.5000000000000001.
Valor esperado: 0.5.
Diferença relativa: 2.22e-14%



Integral de x^2 de 0 a 1
Valor obtido: 0.3350000000000001.
Valor esperado: 0.3333333333333333.
Diferença relativa: 5.00e-01%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.50656.
Valor esperado: 4.4.
Diferença relativa: 2.42e+00%



Integral de sin(x) de 0 a pi
Valor obtido: 1.9835235375094544.
Valor esperado: 2.
Diferença relativa: 8.24e-01%


In [8]:
print("Regra do Trapézio Adaptativa\n")
testeintegral(lambda start, end, func, N: trapezioA(start, end, func, eps=1e-2), 10)

# Em todos os casos o erro é de facto menor que eps!

Regra do Trapézio Adaptativa

Integral de x de 0 a 1
Valor obtido: 0.5.
Valor esperado: 0.5.
Diferença relativa: 0.00e+00%



Integral de x^2 de 0 a 1
Valor obtido: 0.3359375.
Valor esperado: 0.3333333333333333.
Diferença relativa: 7.81e-01%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.402604103088379.
Valor esperado: 4.4.
Diferença relativa: 5.92e-02%



Integral de sin(x) de 0 a pi
Valor obtido: 1.9935703437723395.
Valor esperado: 2.
Diferença relativa: 3.21e-01%


In [9]:
print("Romberg com Trapézio\n")
testeintegral(lambda start, end, func, N: rombergT(start, end, func, eps=1e-2), 10)

# Em todos os casos o erro é de facto menor que eps!

Romberg com Trapézio

Integral de x de 0 a 1
Valor obtido: 0.5.
Valor esperado: 0.5.
Diferença relativa: 0.00e+00%



Integral de x^2 de 0 a 1
Valor obtido: 0.3333333333333333.
Valor esperado: 0.3333333333333333.
Diferença relativa: 0.00e+00%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.4.
Valor esperado: 4.4.
Diferença relativa: 0.00e+00%



Integral de sin(x) de 0 a pi
Valor obtido: 1.9985707318238357.
Valor esperado: 2.
Diferença relativa: 7.15e-02%


In [10]:
from testes import testeintegral2d

print("Regra do Trapézio 2D\n")
testeintegral2d(trapezio2d, (100, 100))

Regra do Trapézio 2D

Integral de x num quadrado com x, y em [0, 1]
Valor obtido: 0.5.
Valor esperado: 0.5.
Diferença relativa: 0.00e+00%



Integral de x^2 + y^2 num quadrado com x, y em [0, 1]
Valor obtido: 0.6667000000000001.
Valor esperado: 0.6666666666666666.
Diferença relativa: 5.00e-03%



Integral de xy + sin(x) + cos(2y) num quadrado com x, y em [0, 1]
Valor obtido: 1.164327421666051.
Valor esperado: 1.1643464075447012.
Diferença relativa: 1.63e-03%



